In [1]:
using Optim
using AutoGrad
using Plots
using DifferentialEquations

In [2]:
default(size = (300, 200)) # plot size

# Generate data

In [3]:
#f1(t, y) = t^2/20 + sin(2*t)
f1(t, y) = -y + sin(t)

f1 (generic function with 1 method)

In [4]:
t = reshape([0.0:0.1:6.0;],1,:) # shape = [n_input,n_sample]

1×61 Array{Float64,2}:
 0.0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  …  5.4  5.5  5.6  5.7  5.8  5.9  6.0

# Solve by DifferentialEquations.jl

In [5]:
prob = ODEProblem(f1, 1.0, (0.0, 6.0))

DiffEqBase.ODEProblem with uType Float64 and tType Float64. In-place: false
timespan: (0.0, 6.0)
u0: 1.0

In [6]:
sol = solve(prob, saveat=0.1);
y_true = sol.u
size(y_true)

(61,)

In [7]:
plot(t[:], y_true, label="y")
plot!(t[:], f1.(t[:] ,y_true), label="dy/dt")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y 
 
 
 
 dy/dt

# Build NN

In [8]:
function init_weights(;n_in=1, n_hidden=10, n_out=1)
    W1 = randn(n_hidden, n_in) # for left multiply W1*x
    b1 = zeros(n_hidden)
    W2 = randn(n_out, n_hidden)
    b2 = zeros(n_out)
    params = [W1, b1, W2, b2]
    return params
end

params = init_weights()

4-element Array{Array{Float64,N} where N,1}:
 [0.0261692; -1.58528; … ; 0.213485; -0.555207]    
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [-1.63155 0.956377 … -0.544057 -1.13117]          
 [0.0]                                             

In [9]:
params_flat = collect(Iterators.flatten(params))
sizes = map(size, params)

4-element Array{Tuple{Int64,Vararg{Int64,N} where N},1}:
 (10, 1)
 (10,)  
 (1, 10)
 (1,)   

In [10]:
function predict(params, t; act=tanh, y0=1.0, t0=0.0)
    
    W1, b1, W2, b2 = params
    
    # normal NN calculation
    a = act.(W1*t .+ b1)
    y = W2*a .+ b2
    
    # force intial condition
    phi = y0 .+ (t .- t0) .* y
    
    return phi
end

predict (generic function with 1 method)

In [11]:
function unflatten(params_flat)
    params = []
    i1 = 1
    for s in sizes # sizes is defined outside of the function
        l = reduce(*, s) # size -> length
        i2 = i1+l
        #p = reshape(params_flat[i1:i2-1], s)
        p = reshape(view(params_flat,i1:i2-1), s)
        push!(params, p)
        i1 = i2
    end 
    return params
end

unflatten (generic function with 1 method)

In [12]:
unflatten(params_flat) == params

true

In [13]:
f_pred = predict(params, t)

1×61 Array{Float64,2}:
 1.0  1.01209  1.04824  1.10798  …  2.04487  1.98924  1.93351  1.8777

In [14]:
plot(t[:], f_pred[:])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

## Grad NN w.r.t to t

Autograd.jl only works with scalar. We can take the diagonal of the jacobian to "vectorize" over t.

In [15]:
predict_sum(params, t) = sum(predict(params, t))

predict_sum (generic function with 1 method)

In [16]:
predict_dt = grad(predict_sum, 2)

(::gradfun) (generic function with 1 method)

In [17]:
dfdt_pred = predict_dt(params, t)

1×61 Array{Float64,2}:
 0.0  0.241581  0.480683  0.712227  …  -0.556833  -0.557723  -0.558501

In [18]:
plot(t[:], dfdt_pred[:])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

## Build loss function

In [19]:
function loss_func(params, t)
    y_pred = predict(params, t)
    dydt_pred = predict_dt(params, t)
    f_pred = f1.(t, y_pred)
    loss = mean(abs2, dydt_pred - f_pred)
    return loss
end

loss_func (generic function with 1 method)

In [20]:
loss_func(params, t)

7.317734966396421

# Optim.jl

Objective function should only take one variable.

In [21]:
# use global t for now
function loss_wrap(params_flat)
    params = unflatten(params_flat)
    return loss_func(params, t)
end

loss_wrap (generic function with 1 method)

In [22]:
loss_wrap(params_flat)

7.317734966396421

## Built-in Forward autodiff

- http://julianlsolvers.github.io/Optim.jl/stable/algo/autodiff/

Seems to speed up the optimizer by 200%

In [23]:
od = OnceDifferentiable(loss_wrap, params_flat; autodiff =:forward);
typeof(od)

NLSolversBase.OnceDifferentiable{Float64,Array{Float64,1},Array{Float64,1},Val{false}}

Do several steps of Momentum Gradient Descent to approach minimum, and then use BFGS to converge to minimum.

In [24]:
option1 = Optim.Options(iterations = 50, show_trace=true, show_every=50)
option2 = Optim.Options(iterations = 100, show_trace=true, show_every=50)

Optim.Options{Float64,Void}(1.0e-32, 1.0e-32, 1.0e-8, 0, 0, 0, false, 100, false, true, false, 50, nothing, NaN)

In [25]:
params = init_weights(n_hidden = 10) # re-initialize weight
params_flat = collect(Iterators.flatten(params));

In [26]:
@time opt = optimize(od, params_flat, MomentumGradientDescent(), option1)
@time opt = optimize(od, opt.minimizer, BFGS(), option2)

Iter     Function value   Gradient norm 
     0     5.086741e+01     2.777280e+02
    50     1.204698e-01     1.785478e-01
  2.047557 seconds (1.56 M allocations: 392.091 MiB, 10.98% gc time)
Iter     Function value   Gradient norm 
     0     1.204698e-01     1.785478e-01
    50     7.977925e-05     1.999047e-02
   100     1.147647e-05     1.355484e-02
  1.243567 seconds (1.21 M allocations: 529.072 MiB, 9.31% gc time)


Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [0.3932750142588691,-0.26899657487302986, ...]
 * Minimizer: [0.3323257181464127,-0.41072395248197135, ...]
 * Minimum: 1.147647e-05
 * Iterations: 100
 * Convergence: false
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 1.12e-02 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = 7.08e-02 
   * |g(x)| < 1.0e-08: false 
     |g(x)| = 1.36e-02 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: true
 * Objective Calls: 450
 * Gradient Calls: 450

In [27]:
loss_wrap(opt.minimizer)

1.1476471779897683e-5

In [28]:
dydt_pred = predict_dt(unflatten(opt.minimizer), t)
plot(t[:], f1.(t[:] ,y_true), label="dy/dt true")
plot!(t[:], dydt_pred[:],lw=0,label="dy/dt predict",
    marker=:circle,markerstrokewidth = 0,markersize=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 -1.0 
 
 
 -0.8 
 
 
 -0.6 
 
 
 -0.4 
 
 
 -0.2 
 
 
 0.0 
 
 
 0.2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 dy/dt true 
 
 
 
 
 dy/dt predict

In [29]:
plot(t[:], y_true, label="y true")
y_pred = predict(unflatten(opt.minimizer), t)
plot!(t[:], y_pred[:],lw=0,label="y predict",
     marker=:circle,markerstrokewidth = 0,markersize=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y true 
 
 
 
 
 y predict